In [8]:
import numpy as np
import pandas as pd
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import RNN
from sklearn.model_selection import train_test_split
from keras.layers import GRU
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

**Read data**

In [9]:
import pandas as pd
df = pd.read_csv("/Users/rahuljauhari/Desktop/research-runoff/Data/merged_imd.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.iloc[:, :157]

df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()
print(monthly_mean.shape)

df_actual = pd.read_excel("/Users/rahuljauhari/Desktop/research-runoff/Data/Calibrated and Validated.xlsx")
# select last column
observed_runnoff = df_actual['observed']
# observed_runnoff.head()
print(observed_runnoff.shape)

(468, 156)
(468,)


**Normalization**

In [16]:
from scipy.stats import zscore
def func(name):
    x=0
    y=0
    inv= 0
    if name=='zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm >3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x=x_norm
        y=y_norm
    if name=='StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled >3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x=      x_scaled  
        y=y_scaled
        inv = scaler
        
    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0,1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1,1))
        x=      x_scaled  
        y=y_scaled
        inv = scaler
    return x,y,inv


In [17]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

In [18]:
preprocessing = ['StandardScaler']
models = [SimpleRNN]
activations =['tanh']
optimizers = ['adam']

In [19]:
for pre in preprocessing:
    x,y,inv_scaler= func(pre)
    X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)
    for mod in models:
        for act in activations:
            for opt in optimizers:
                print('preprocessing:',pre,'model:',mod,'activation:',act,'optimizer:',opt)
                model = Sequential()
                model.add(mod(64,return_sequences=True, input_shape=(X_train.shape[1], 1),activation=act))
                model.add(mod(64,activation=act))  
                model.add(Dense(1,activation=act))
                model.compile(loss='mean_squared_error', optimizer=opt)
                model.fit(x,y,epochs=100,batch_size=10,verbose=0, validation_split=0.1,shuffle=False)
                y_pred_test=model.predict(X_test)
                y_pred_train=model.predict(X_train)
                # try:
                #     _ = pd.DataFrame({'model':mod,'activation':act,'optimizer':opt,'preprocessing':pre,'train rmse':rmse1(y_train,y_pred_train),'test rmse':rmse1(y_test,y_pred_test),'train kge':kge1(y_train,y_pred_train),'test kge':kge1(y_test,y_pred_test),'train r2':r21(y_train,y_pred_train),'test r2':r21(y_test,y_pred_test)},index=['model'])
                #     _.to_csv('/Users/rahuljauhari/Desktop/research runoff/nasa/imd_result.csv',mode='a',header=True)
                # except:
                #     print('error') 

preprocessing: StandardScaler model: <class 'keras.layers.rnn.simple_rnn.SimpleRNN'> activation: tanh optimizer: adam


In [ ]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred_train_inv = inv_scaler.inverse_transform(y_pred_train)
y_pred_test_inv = inv_scaler.inverse_transform(y_pred_test)
y_train__inv = observed_runnoff[:len(y_pred_train_inv)]
y_test__inv = observed_runnoff[len(y_pred_train_inv):]
print("KGE train: ", round(kge1(y_train__inv, y_pred_train_inv),4))
print("KGE test: ", round(kge1(y_test__inv, y_pred_test_inv),4))



In [ ]:
y_train__inv = pd.DataFrame(y_train__inv)
y_pred_train_inv = pd.DataFrame(y_pred_train_inv)
y_train__inv.reset_index(drop=True, inplace=True)
y_pred_train_inv.reset_index(drop=True, inplace=True)
y_train__inv = pd.concat([y_train__inv,y_pred_train_inv],axis=1)
y_train__inv.to_csv(os.getcwd()+'train_rnn_0.2.csv',mode='a',header=True)
y_test__inv = pd.DataFrame(y_test__inv)
y_pred_test_inv = pd.DataFrame(y_pred_test_inv)
y_test__inv.reset_index(drop=True, inplace=True)
y_pred_test_inv.reset_index(drop=True, inplace=True)
y_test__inv = pd.concat([y_test__inv,y_pred_test_inv],axis=1)
y_test__inv.to_csv(os.getcwd()+'test_rnn_0.2.csv',mode='a',header=True)

**Train test split**

In [ ]:
# import train test split
from sklearn.model_selection import train_test_split
x,y= func('StandardScaler')
X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,shuffle=False)

In [ ]:
l=[]
for i in range(0,observed_runnoff.shape[0]):
    l.append(i)
plt.scatter(l,y)
plt.show()

Model

In [ ]:
model1 = Sequential()
model1.add(SimpleRNN(64,return_sequences=True, input_shape=(X_train.shape[1], 1),activation='tanh'))
# model1.add(Dropout(0.2))      
model1.add(SimpleRNN(64,activation='tanh'))  
model1.add(Dense(1,activation='tanh'))
model1.compile(optimizer="adam", loss='mse')
model1.summary()

In [ ]:
model1.fit(X_train, y_train,validation_split=0.1, batch_size=10, epochs=100,shuffle=False, use_multiprocessing=True)

**Metric**

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse1(yt, yp): #lower the better
    return np.sqrt(mean_squared_error(yt, yp))
# Kling-Gupta effciency
def kge1(yt, yp): #highqer the better
    r = np.corrcoef(yt, yp,rowvar=False)[0, 1]
    alpha = np.std(yp) / np.std(yt)
    beta = np.mean(yp) / np.mean(yt)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
# Normalized standard Error 
def nse1(yt, yp): 
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)
    # r squared
def r21(yt, yp): #higher the better
    return 1 - np.sum((yt - yp)**2) / np.sum((yt - np.mean(yt))**2)

**Train**

In [ ]:
yp1 = model1.predict(X_train)

In [ ]:
yp1[0:10]

In [ ]:
y_train[0:10]

In [ ]:
kge = []
r2=[]
rmse =[]
# for i in range(yp1.shape[]):
kge.append(kge1(y_train, yp1))
r2.append(r21(y_train, yp1))
rmse.append(rmse1(y_train, yp1))

In [ ]:
max(r2)

In [ ]:
min(rmse)

In [ ]:
max(kge)

**Test**

In [ ]:
yp1 = model1.predict(X_test)

In [ ]:
yp1[0:10]

In [ ]:
y_test[0:10]

In [ ]:
kge = []
r2=[]
rmse =[]
print(yp1.shape)
# for i in range(yp1.shape[]):
kge.append(kge1(y_test, yp1))
r2.append(r21(y_test, yp1))
rmse.append(rmse1(y_test, yp1))

In [ ]:
max(r2)

In [ ]:
min(rmse)

In [ ]:
max(kge)